In [1]:
from pathlib import Path
from tqdm import tqdm

import json
import yaml

In [2]:
def dict2json(judg,path):
    file = open(path,'w')
    json.dump(judg,file)
    file.close()

def get_gpath_dict():
    paths = yaml.safe_load(open('/home2/sisodiya.bhoomendra/github/contrastive_learning/data/paths.yaml','r'))
    posix_path = {}
    for k,v in paths.items():
        posix_path[k] = Path(v)
    return posix_path

Load the data

In [3]:
x = get_gpath_dict()
x

{'path_html_judg': PosixPath('/home2/sisodiya.bhoomendra/judgments'),
 'path_home': PosixPath('/home2/sisodiya.bhoomendra'),
 'path_scratch': PosixPath('/scratch/contrastive'),
 'path_clean': PosixPath('../data/processed/processed_judgements'),
 'path_bert_ready': PosixPath('../data/processed/bert_ready')}

In [4]:
def get_all_paths(path):
    print("Collecting all paths")
    paths = []
    for fpath in path.iterdir():
        paths.append(fpath)
    return paths

In [6]:
paths = get_all_paths(x['path_clean'])
file = open(paths[0],'r')
judg = json.load(file)
file.close()

In [7]:
out = list()
for path in tqdm(paths,desc="Link extraction"):
    file = open(path,'r')
    judg = json.load(file)
    out.append([path.name.split('.')[0],judg['citation_sc']]) 
    file.close()

Link extraction: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 53897/53897 [00:32<00:00, 1641.93it/s]


In [8]:
nodes = [x[0] for x in out]

In [9]:
len(nodes)

53897

In [10]:
edges = []
for x in tqdm(out,desc="edges"):
    for y in x[1]:
        edges.append((x[0],y))

edges: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 53897/53897 [00:00<00:00, 695583.31it/s]


In [11]:
len(edges)

132984

In [12]:
import networkx as nx
G = nx.DiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [13]:
print(G)

DiGraph with 53897 nodes and 132984 edges


In [73]:
def get_dist_2(source):
    out = list(nx.bfs_edges(G, source=source, depth_limit=2))
    dist1 =  set([ x[0] for x in out if x[0]!=source ])
    triplet = [ (source,x[0],x[1]) for x in out if x[0] in dist1 ]
    return triplet

In [53]:
import re
IRLED_map = open('../data/maps.txt')
text = IRLED_map.read()
idx = re.findall('\d+.json',text)

In [54]:
len(idx)

200

In [56]:
idx[0][:-5]

'665756'

In [57]:
idx = [ x.split('.')[0] for x in idx]

In [59]:
idx = set(idx)

In [60]:
AILA_2019 = open('../../para_based_legal_search/data/raw/query2indiankanoon.txt','r')
text = AILA_2019.read()

In [63]:
aila_ids = [ x.split()[-1]  for x in  text.split('\n')]

In [66]:
aila_ids = set(aila_ids)

In [70]:
len(aila_ids)

22

In [68]:
for x in aila_ids:
    if x not in 'NAN':
        idx.add(x)

In [69]:
len(idx)

218

In [75]:
pos = []
neg = []
for node in tqdm(nodes,desc='Making constrastive pairs'):
    # print(node)
    if node not in idx:
        spos,sneg = get_dist_2(node)
        pos.extend(spos)
        neg.extend(sneg)

Making constrastive pairs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 53897/53897 [00:00<00:00, 67543.06it/s]


In [76]:
len(pos)

130446

In [77]:
len(neg)

652356